In [1]:
import pandas as pd
import scipy.stats as stats
import os

Concatenate our files. Reminder: we have a file per year, we all of 'Informatique' in it.

In [3]:
# requires os
def concatFiles(direc, fileType):
    files = os.listdir(direc)
    if '.DS_Store' in files:
        files.remove('.DS_Store')
    print(files)
    if fileType == 'csv':
        r = pd.read_csv
    for idx, file in enumerate(files):
        file = direc+file
        if idx == 0:
            df = r(file, header=0)
            print(file)
        else:
            new = r(file, header=0)
            df = pd.concat([df, new], axis=0)
            print(file)
    return df

In [4]:
info = concatFiles('data/', 'csv')

['informatique_2011_2012.csv', 'informatique_2014_2015.csv', 'informatique_2012_2013.csv', 'informatique_2008_2009.csv', 'informatique_2007_2008.csv', 'informatique_2016_2017.csv', 'informatique_2015_2016.csv', 'informatique_2010_2011.csv', 'informatique_2013_2014.csv', 'informatique_2009_2010.csv']
data/informatique_2011_2012.csv
data/informatique_2014_2015.csv
data/informatique_2012_2013.csv
data/informatique_2008_2009.csv
data/informatique_2007_2008.csv
data/informatique_2016_2017.csv
data/informatique_2015_2016.csv
data/informatique_2010_2011.csv
data/informatique_2013_2014.csv
data/informatique_2009_2010.csv


We define a locating function to grab only bachelor data

In [8]:
def locator(s):
    return s.find('Bachelor') != -1

In [11]:
ba = info.loc[info['Semester'].apply(locator)]

We define a function to locate a specific sciper #. They are unique.

In [12]:
def locSciper(df, sciper):
    return df.loc[df['No Sciper'] == sciper]

isOneSix checks if there is an entry for BA1 and BA6 in the given DataFrame

In [13]:
def isOneSix(df):
    """ Take a DataFrame and check that there is an entry for both BA1 and BA6
    
    Args:
        df (DataFrame): a DataFrame, typically a .loc on a specific student
        
    Returns:
        bool: True if it finds both BA1 and BA6, False otherwise
    """
    try:
        one = df.isin(['Bachelor semestre 1'])['Semester'].value_counts()[True] > 0
    except KeyError:
        one = 0
    try:
        six = df.isin(['Bachelor semestre 6'])['Semester'].value_counts()[True] > 0
    except KeyError:
        six = 0
    return (one and six)

In [14]:
def getGender(df):
    """ Take a DataFrame and checks the gender
    
    Args:
        df (DataFrame): a DataFrame, typically a .loc on a specific student
        
    Returns:
        bool: True if student is a woman, False otherwise
    """
    try:
        if df.isin(['Madame'])['Civilité'].value_counts()[True] > 0:
            return 1
    except KeyError:
        return 0

We loop through the bachelor rows, perform checks and fill a dict with appropriate data.

In [16]:
dico = {'sciper': [], 'gender': [], 'length': []}
for row in ba.iterrows():
    o = row[1]
    sciper = o['No Sciper']
    # Making sure we consider unique sciper #s
    if not sciper in dico['sciper']:
        df = locSciper(ba, sciper)
        if isOneSix(df):
            dico['sciper'].append(sciper)
            dico['gender'].append(getGender(df))
            # Calculating length of stay by nbr of rows
            dico['length'].append(len(df))

We transform this dict into a DataFrame, now containing only the students we're concerned with

In [17]:
data = pd.DataFrame(dico)

We do a quick describe to check for inconsistencies

In [180]:
data['length'].describe()

count    398.000000
mean       7.080402
std        1.523475
min        4.000000
25%        6.000000
50%        6.000000
75%        8.000000
max       12.000000
Name: length, dtype: float64

We notice the minimum is 4, which doesn't seem to make sense. So we check what it is:

In [182]:
data.loc[data.length < 6]

,gender,length,sciper
241,0,4,204222


In [19]:
locSciper(ba, 204222)

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Subject,Period,Semester
165,Monsieur,Séguy Louis Marie James,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,204222,Informatique,2011-2012,Bachelor semestre 1
290,Monsieur,Séguy Louis Marie James,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,204222,Informatique,2011-2012,Bachelor semestre 2
721,Monsieur,Séguy Louis Marie James,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,204222,Informatique,2014-2015,Bachelor semestre 5
840,Monsieur,Séguy Louis Marie James,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,204222,Informatique,2014-2015,Bachelor semestre 6


It seems there are missing records for this person. We could take him out.

Setting up our statistical calculations

In [20]:
men = data.loc[data.gender == 0]

In [21]:
men.length.mean()

7.102981029810298

In [22]:
women = data.loc[data.gender == 1]

In [23]:
women.length.mean()

6.793103448275862

In [24]:
stats.ttest_ind(a=women.length, b=men.length, )

Ttest_indResult(statistic=-1.0548414519194305, pvalue=0.29214098486369383)

Not significant.